 Like probably many of you, I was *very* surprised when I saw the final results this morning. As the discussion [here](https://www.kaggle.com/c/seti-breakthrough-listen/discussion/266385) has shown, the key to the large distance between the winning solution and #2 was what the winning team called "magic #2". This method allowed them to remove background noise completely for at least some samples. Achieving this would be a kind of Holy Grail for experimental physics, so I really wanted to understand this method. At first, I couldn't wrap my head aroungd their explanation, so I started to reimplement it on my own.
 
DISCLAIMER: even though I think that in the end it is some kind of leak, the winning team's solution is absolutely brilliant. I would have never figured this out on my own. And of course, I'm not 100% sure that this is all there is to "magic #2". This is just my attempt at an explanation.
 
By now, the following is clear to me:

The key is that the true measured background signals used in this competition are far wider in the frequency range than 256 bins. Therefore, the organizers cut the data up into bands of 256 bins each and used these as samples. However, for reasons unclear to me they user *severely* overlapping samples. Therefore, for most samples in the datasets, you can -partially- find that data again in another sample. If one of them contains a needle, you can perfectly remove any noise.
The only remaining problem is the fact that each image in a sample is independently normalized to mean=0, std=1. This needs to be reversed.

I will demonstrate this for one sample:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
INPUT_DIR = "../input/seti-breakthrough-listen"
df_train = pd.read_csv(os.path.join(INPUT_DIR, "train_labels.csv"))
df_subm = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))
df_train_pos = df_train[df_train.target == 1]
df_train_neg = df_train[df_train.target == 0]

Helper functions for loading and displaying samples:

In [ ]:
def load_example(idx):
    try:
        x = np.load(os.path.join(INPUT_DIR, "train", idx[0], idx + ".npy"))
    except:
        x = np.load(os.path.join(INPUT_DIR, "test",  idx[0], idx + ".npy"))
    return np.take(x.astype(np.float32), [0,2,4], 0)


def show_example(x, p=0):
    x = x.reshape(-1, 256)
    x = np.clip(x, np.percentile(x, p), np.percentile(x, 100-p)) # clip for better contrast
    
    fig, ax = plt.subplots()
    fig.set_size_inches(9, 9)
    ax.set_xticks(np.arange(1,3)*273)
    ax.set_yticks([])
    ax.grid(True)
    ax.imshow(x.T, aspect="auto", cmap="Greys")

Let's take a look at one sample which contains a needle with a very low SNR, definitely not visible by eye, the 5th positive sample in train. Time is the horizontal direction, frequency vertical:

In [ ]:
idx = df_train_pos.id.iloc[5]
print(idx)
x0 = load_example(idx)
show_example(x0, 1)

I'm doing the "renormalization" similar to but probably different from the winning team. I start with one normalized reference column and check if this column is found somewhere else in the data:

In [ ]:
def column_norm(x):
    ''' normalizes each column of 273 pixels in each of the 6 images in sample x separately 
    to mean==0 and L2 norm==1 '''
    
    xn  = x - np.mean(x, axis=1, keepdims=True) # remove mean
    xn /= np.sqrt(np.sum(xn to **2, axis=1, keepdims=True)) # normalize
    return xn  
def find_similar_column_old(col0, xn1):
    ''' calculates cosine similarity between the normalized reference column col0 and all columns
        in the column-normalized sample xn1 '''
    
    return np.array([ [ np.dot(col0, col1) for col1 in img.T ] for img in xn1 ])

def find_similar_column(image0, image1):
    ''' calculates shift of image '''
    middle_column = int(len(image1)/2)
    sums = [0] * len(image1[middle_column])
    #for (z, col1) in enumerate(image1):
    col1 = image1[middle_column]
    col0 = image0[middle_column]
    for (i, data) in enumerate(col1):
        col1_rolled = np.roll(col1, i, axis=0)
        result = np.sum(np.square(col0 - col1_rolled))
        sums[i] += result
            #bestMatch.append((i, result))  
    return (sums.index(min(sums)), min(sums))
                
    #return np.array([ [ np.dot(col0, col1) for col1 in img.T ] for img in xn1 ])

The normalized sample from above. Still no needle visible

Now let's search the dataset for a copy of normalized column 128 in image 1 of our sample. I use col 128 because it's in the middle of the image. I search only a small subset of the full 60,000 samples dataset to keep running time short because I know from earlier runs where the match will be ;).

## Search-algorithm

In [ ]:
import pandas as pd
import csv
best_matches = []
for idy in tqdm(df_train_pos.id.iloc[10:30]):
    css = []
    x0 = load_example(idy)
    xn0 = column_norm(x0)
    for idx in tqdm(df_train.id.iloc[0:df_train.size]):
        if idx == idy:
            print(idx)
        else:            
            xn1 = column_norm(load_example(idx))
            cs = find_similar_column_old(xn0[0,:,128], xn1)
            csm = cs.max()
            css.append((idx, csm, cs.argmax() % 273))
    largest = max(css, key = lambda t: t[1])
    best_matches.append({"search": idy, "match": largest})
with open('outputfile.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for item in best_matches:        
        for key, value in item.items():
           writer.writerow([key, value])
print(best_matches)

In [ ]:
largest = max(css, key = lambda t: t[1])
css.remove(largest)
largest_2 = max(css, key = lambda t: t[1])
print(largest)
print(largest_2)

In [ ]:
x1 = load_example(largest_2[0])
xn1 = column_norm(x1)
#result = find_similar_column(xn0[0,100:150], xn1[0,100:150])
#print(result)

Found a perfect match: sample d87cb86179e9d02. Let's look at it. It is identical to 004933b94083be2 where they overlap It's only shifted by 128-68=60 frequency bins:

#5: d87cb86179e9d02

#12: b1f73e62e285d45

In [ ]:
search_list = [{'search': '0024012d1431fbc', 'match': ('fa59fd3c6615824', 0.99999946, 14)}, 
{'search': '0028a35de92941d', 'match': ('e26ddbf40e0f281', 0.9393103, 26)}, 
{'search': '002efdabe4e3e45', 'match': ('e77d17e31e82c38', 0.9999997, 173)}, 
{'search': '0031e823c133be2', 'match': ('5bf8895f9bb6efb', 0.9079099, 128)}, 
{'search': '00412077d1aef6f', 'match': ('7fe84991f5f6808', 0.94292027, 252)}, 
{'search': '004933b94083be2', 'match': ('d87cb86179e9d02', 0.9999998, 68)}, 
{'search': '0055e9458a0f03a', 'match': ('390f9877282e72c', 0.48137662, 165)}, 
{'search': '00776881dd80050', 'match': ('9566e73eab8d403', 0.9999927, 241)}, 
{'search': '0080e5e1ef92b4c', 'match': ('4b3be228e8f66b0', 0.876866, 21)}, 
{'search': '008eb601300b642', 'match': ('008eb601300b642', 0.9999999, 128)}]

In [ ]:
for item in search_list:
    print(item)
    if (item['match'][1] > 0.95):
        x0 = load_example(item['search']) 
        xn0 = column_norm(x0)
        x1 = load_example(item['match'][0])
        xn1 = column_norm(x1)
        result = find_similar_column(xn0[0,:], xn1[0,:])
        xn1_rolled = np.roll(xn1, result[0], axis=2)
        show_example(xn0, 20) 
        show_example(xn1_rolled, 20) 
        show_example(xn1_rolled - xn0, 20)    

In [ ]:
#xn0 = np.swapaxes(xn0,1,2)
#xn1 = np.swapaxes(xn1,1,2)
print(xn0.shape)
print(xn1.shape)
show_example(xn0, 1)
show_example(xn1, 1)

In [ ]:
results = [100]
iteration = 0
for i in range(280):
    test = xn0 - np.roll(xn1, i, axis=2)
    cos = np.array([ [ np.dot(xn0[0,:,128], col1) for col1 in img.T ] for img in test ]).max()
    results.append(cos)
    if cos < results[iteration]:
        iteration = i
print(iteration)
print(results[60])
print(results[127])

Normalize it and subtract from the normalized sample xn0. Increasing the contrast a little, the needles are easily visible. The noise has been perfectly removed because of the leak. A 2-layer MLP could find these needles:

In [ ]:
show_example(xn0)
show_example(np.roll(xn1, 60, axis=2))

Increasing the contrast some more, numerical rounding errors become visible, except for where the artificial signals were inserted. Obviously using rectangles.

In [ ]:
show_example(xn0 - np.roll(xn1, 128-68, axis=2), 40)

That's all for now. Hope this clears up the magic a bit.
Again: congrats to Team Watercooled. Brilliant detective work!